In [163]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error as MSE
from glob import glob
import random
import pickle
from tqdm.auto import tqdm

In [127]:
data_files = glob("../../../zadanie2_7z/hmg/joined/*.csv")

In [114]:
#split train / val 85% / 15 %
random.seed(234)
val_split = round(len(data_files)*0.15)
val_files = random.sample(data_files, val_split)
train_files = [x for x in data_files if x not in val_files]

In [61]:
def split_seq(data_x, data_y, steps):
    x, y = list(), list()
    for i in range(len(data_x) - steps):
        end = i + steps
        seq_x = data_x[i:end, :]
        seq_y = data_y[end, :]
        x.append(seq_x)
        y.append(seq_y)
    return np.array(x), np.array(y)

In [172]:
val_files

['../../../zadanie2_7z/hmg/joined/data_2021-05-23.csv',
 '../../../zadanie2_7z/hmg/joined/data_2021-06-06.csv',
 '../../../zadanie2_7z/hmg/joined/data_2021-05-22.csv',
 '../../../zadanie2_7z/hmg/joined/data_2021-05-16.csv',
 '../../../zadanie2_7z/hmg/joined/data_2021-04-23.csv',
 '../../../zadanie2_7z/hmg/joined/data_2021-04-27.csv']

In [130]:
dane = pd.read_csv(train_files[0]).drop(columns = ['Unnamed: 0'])
dane_y = dane.drop(columns=['001FCx00285_SPPV.PV', '001XXXCALC01.NUM.PV[3]', '001SCx00274_SPPV.PV', '001FCx00241_sppv.pv']).to_numpy()
dane_x = dane.to_numpy()

In [164]:
# exp 1 _ steps = 5

n_steps = 5
x, y = split_seq(dane_x, dane_y, steps = n_steps)
x_features = x.shape[2]
y_features = y.shape[1]

model = Sequential()
model.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(n_steps, x_features)))
model.add(LSTM(100, activation='relu', return_sequences=True))
model.add(LSTM(100, activation='relu'))
model.add(Dense(y_features))
model.compile(optimizer='adam', loss='mse')

for file in tqdm(train_files):
    dane = pd.read_csv(file).drop(columns = ['Unnamed: 0'])
    dane_y = dane.drop(columns=['001FCx00285_SPPV.PV', '001XXXCALC01.NUM.PV[3]', '001SCx00274_SPPV.PV', '001FCx00241_sppv.pv']).to_numpy()
    dane_x = dane.to_numpy()
    x, y = split_seq(dane_x, dane_y, steps = n_steps)
    history = model.fit(x, y, epochs = 5, verbose = 1)
model.save('../../../zadanie2_7z/hmg/results/exp1/model.h5')
with open('../../../zadanie2_7z/hmg/results/exp1/history.txt', 'wb') as file_pi:
    pickle.dump(history.history, file_pi)

Epoch 1/5
2700/2700 [==============================] - 40s 13ms/step - loss: 2333522.7312
Epoch 2/5
2700/2700 [==============================] - 34s 13ms/step - loss: 3124.8970
Epoch 3/5
2700/2700 [==============================] - 34s 13ms/step - loss: 2495.1301
Epoch 4/5
2700/2700 [==============================] - 33s 12ms/step - loss: 2268.0322
Epoch 5/5
2700/2700 [==============================] - 33s 12ms/step - loss: 4008.5493
Epoch 1/5
2700/2700 [==============================] - 33s 12ms/step - loss: 9148.8838
Epoch 2/5
2700/2700 [==============================] - 34s 13ms/step - loss: 2054.7576
Epoch 3/5
2700/2700 [==============================] - 34s 13ms/step - loss: 1778.6422
Epoch 4/5
2700/2700 [==============================] - 33s 12ms/step - loss: 1686.2064
Epoch 5/5
2700/2700 [==============================] - 33s 12ms/step - loss: 1604.9684
Epoch 1/5
2700/2700 [==============================] - 32s 12ms/step - loss: 3375.4087
Epoch 2/5
2700/2700 [===================

In [168]:
import keras

In [170]:
model = keras.models.load_model('../../../zadanie2_7z/hmg/results/exp2/model1.h5')

In [ ]:
# exp 2 _ steps = 5, simpler model

n_steps = 5
x, y = split_seq(dane_x, dane_y, steps = n_steps)
x_features = x.shape[2]
y_features = y.shape[1]

#model = Sequential()
#model.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(n_steps, x_features)))
#model.add(LSTM(100, activation='relu', return_sequences=True))
#model.add(LSTM(100, activation='relu'))
#model.add(Dense(y_features))
#model.compile(optimizer='adam', loss='mse')
hist = []
for file in tqdm(train_files):
    dane = pd.read_csv(file).drop(columns = ['Unnamed: 0'])
    dane_y = dane.drop(columns=['001FCx00285_SPPV.PV', '001XXXCALC01.NUM.PV[3]', '001SCx00274_SPPV.PV', '001FCx00241_sppv.pv']).to_numpy()
    dane_x = dane.to_numpy()
    x, y = split_seq(dane_x, dane_y, steps = n_steps)
    history = model.fit(x, y, epochs = 3, verbose = 1)
    hist.append(history.history)
    model.save('../../../zadanie2_7z/hmg/results/exp2/model_last.h5')
model.save('../../../zadanie2_7z/hmg/results/exp2/model.h5')
with open('../../../zadanie2_7z/hmg/results/exp2/history.txt', 'wb') as file_pi:
    pickle.dump(hist, file_pi)

Epoch 1/3
2700/2700 [==============================] - 37s 13ms/step - loss: 208.4284
Epoch 2/3
2700/2700 [==============================] - 34s 13ms/step - loss: 163.6166
Epoch 3/3
2700/2700 [==============================] - 34s 13ms/step - loss: 158.1809
Epoch 1/3
2700/2700 [==============================] - 34s 13ms/step - loss: 131.8605
Epoch 2/3
2700/2700 [==============================] - 35s 13ms/step - loss: 136.0431
Epoch 3/3
2700/2700 [==============================] - 34s 13ms/step - loss: 142.9281
Epoch 1/3
2700/2700 [==============================] - 34s 13ms/step - loss: 602.3824
Epoch 2/3
2700/2700 [==============================] - 33s 12ms/step - loss: 290.4125
Epoch 3/3
2700/2700 [==============================] - 33s 12ms/step - loss: 246.2331
Epoch 1/3
1800/1800 [==============================] - 23s 13ms/step - loss: 177.4799
Epoch 2/3
1800/1800 [==============================] - 23s 13ms/step - loss: 130.7615
Epoch 3/3
1800/1800 [==============================] -

In [ ]:
# exp 1 _ steps = 10

n_steps = 10
x, y = split_seq(dane_x, dane_y, steps = n_steps)
x_features = x.shape[2]
y_features = y.shape[1]

model = Sequential()
model.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(n_steps, x_features)))
model.add(LSTM(100, activation='relu', return_sequences=True))
model.add(LSTM(100, activation='relu'))
model.add(Dense(y_features))
model.compile(optimizer='adam', loss='mse')

for file in train_files:
    dane = pd.read_csv(train_files[0]).drop(columns = ['Unnamed: 0'])
    dane_y = dane.drop(columns=['001FCx00285_SPPV.PV', '001XXXCALC01.NUM.PV[3]', '001SCx00274_SPPV.PV', '001FCx00241_sppv.pv']).to_numpy()
    dane_x = dane.to_numpy()
    x, y = split_seq(dane_x, dane_y, steps = n_steps)
    history = model.fit(x, y, epochs = 400, verbose = 0)
model.save('../../../zadanie2_7z/hmg/results/exp3/model.h5')
with open('../../../zadanie2_7z/hmg/results/exp3/history.txt', 'wb') as file_pi:
    pickle.dump(history.history, file_pi)

In [ ]:
# exp 4 _ steps = 30

n_steps = 30
x, y = split_seq(dane_x, dane_y, steps = n_steps)
x_features = x.shape[2]
y_features = y.shape[1]

model = Sequential()
model.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(n_steps, x_features)))
model.add(LSTM(100, activation='relu', return_sequences=True))
model.add(LSTM(100, activation='relu'))
model.add(Dense(y_features))
model.compile(optimizer='adam', loss='mse')

for file in train_files:
    dane = pd.read_csv(train_files[0]).drop(columns = ['Unnamed: 0'])
    dane_y = dane.drop(columns=['001FCx00285_SPPV.PV', '001XXXCALC01.NUM.PV[3]', '001SCx00274_SPPV.PV', '001FCx00241_sppv.pv']).to_numpy()
    dane_x = dane.to_numpy()
    x, y = split_seq(dane_x, dane_y, steps = n_steps)
    history = model.fit(x, y, epochs = 400, verbose = 0)
model.save('../../../zadanie2_7z/hmg/results/exp4/model.h5')
with open('../../../zadanie2_7z/hmg/results/exp4/history.txt', 'wb') as file_pi:
    pickle.dump(history.history, file_pi)

In [ ]:
# exp 5 _ steps = 60

n_steps = 60
x, y = split_seq(dane_x, dane_y, steps = n_steps)
x_features = x.shape[2]
y_features = y.shape[1]

model = Sequential()
model.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(n_steps, x_features)))
model.add(LSTM(100, activation='relu', return_sequences=True))
model.add(LSTM(100, activation='relu'))
model.add(Dense(y_features))
model.compile(optimizer='adam', loss='mse')

for file in train_files:
    dane = pd.read_csv(train_files[0]).drop(columns = ['Unnamed: 0'])
    dane_y = dane.drop(columns=['001FCx00285_SPPV.PV', '001XXXCALC01.NUM.PV[3]', '001SCx00274_SPPV.PV', '001FCx00241_sppv.pv']).to_numpy()
    dane_x = dane.to_numpy()
    x, y = split_seq(dane_x, dane_y, steps = n_steps)
    history = model.fit(x, y, epochs = 400, verbose = 0)
model.save('../../../zadanie2_7z/hmg/results/exp5/model.h5')
with open('../../../zadanie2_7z/hmg/results/exp5/history.txt', 'wb') as file_pi:
    pickle.dump(history.history, file_pi)

In [ ]:
# exp 6 _ steps = 30

n_steps = 30
x, y = split_seq(dane_x, dane_y, steps = n_steps)
x_features = x.shape[2]
y_features = y.shape[1]

model = Sequential()
model.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(n_steps, x_features)))
model.add(LSTM(100, activation='relu', return_sequences=True))
model.add(LSTM(100, activation='relu', return_sequences=True))
model.add(LSTM(100, activation='relu'))
model.add(Dense(y_features))
model.compile(optimizer='adam', loss='mse')

for file in train_files:
    dane = pd.read_csv(train_files[0]).drop(columns = ['Unnamed: 0'])
    dane_y = dane.drop(columns=['001FCx00285_SPPV.PV', '001XXXCALC01.NUM.PV[3]', '001SCx00274_SPPV.PV', '001FCx00241_sppv.pv']).to_numpy()
    dane_x = dane.to_numpy()
    x, y = split_seq(dane_x, dane_y, steps = n_steps)
    history = model.fit(x, y, epochs = 400, verbose = 0)
model.save('../../../zadanie2_7z/hmg/results/exp6/model.h5')
with open('../../../zadanie2_7z/hmg/results/exp6/history.txt', 'wb') as file_pi:
    pickle.dump(history.history, file_pi)

In [ ]:
# exp 7 _ steps = 20

n_steps = 30
x, y = split_seq(dane_x, dane_y, steps = n_steps)
x_features = x.shape[2]
y_features = y.shape[1]

model = Sequential()
model.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(n_steps, x_features)))
model.add(LSTM(100, activation='relu', return_sequences=True))
model.add(LSTM(100, activation='relu', return_sequences=True))
model.add(LSTM(100, activation='relu'))
model.add(Dense(y_features))
model.compile(optimizer='adam', loss='mse')

for file in train_files:
    dane = pd.read_csv(train_files[0]).drop(columns = ['Unnamed: 0'])
    dane_y = dane.drop(columns=['001FCx00285_SPPV.PV', '001XXXCALC01.NUM.PV[3]', '001SCx00274_SPPV.PV', '001FCx00241_sppv.pv']).to_numpy()
    dane_x = dane.to_numpy()
    x, y = split_seq(dane_x, dane_y, steps = n_steps)
    history = model.fit(x, y, epochs = 500, verbose = 0)
model.save('../../../zadanie2_7z/hmg/results/exp7/model.h5')
with open('../../../zadanie2_7z/hmg/results/exp7/history.txt', 'wb') as file_pi:
    pickle.dump(history.history, file_pi)

In [75]:
dane_y = dane.drop(columns=['001FCx00285_SPPV.PV', '001XXXCALC01.NUM.PV[3]', '001SCx00274_SPPV.PV', '001FCx00241_sppv.pv']).to_numpy()

dane_x = dane.to_numpy()

In [76]:
input_x, input_y = split_seq(dane_x, dane_y, steps = n_steps)

In [77]:
y_pred = model.predict(input_x, verbose=0)

In [80]:
MSE(input_y, y_pred)

3626.4536384230046

In [82]:
input_y[1]

array([2.33180008e+01, 8.70855033e-01, 1.51373994e+00, 7.21114254e+00,
       7.29695272e+00, 6.44669533e+00, 1.14000000e+02, 1.00000000e+01,
       3.00000000e+00, 3.49889832e+02, 3.12519609e+04, 9.90577539e+03,
       2.99000000e+02, 8.68000031e+00, 1.12200003e+01, 2.27999997e+00,
       6.30000000e+01, 1.37000000e+03, 1.36572247e+01, 9.36909103e+00,
       1.31000000e+03, 2.46942893e-01])

In [83]:
y_pred[1]

array([ 2.1546745e+01,  7.4828500e-01, -6.0043967e-01,  3.5830178e+00,
        5.5966163e+00,  1.3751291e+01,  1.1395613e+02,  1.1289093e+01,
        9.4705683e-01,  3.4296664e+02,  3.1261605e+04,  9.8865430e+03,
        2.9139093e+02,  1.1230208e+01,  8.0696602e+00,  6.9680538e+00,
        6.8245575e+01,  1.3641378e+03,  9.3218222e+00,  8.8075914e+00,
        1.2924592e+03,  4.1678696e+00], dtype=float32)